In [2]:
import numpy as np
from matplotlib import pyplot as plt

In [3]:
step = 20
nu = 0.3
ro_v = 1.2
c = 330
plate = 1
plate1 = 0
# vibra_space
#eigen_freq
#acoust_space
#upr_acoust
#diff  - ?
#non_uni

In [4]:
shell = 0 # рассчитываем оболочку или нет
R = np.inf # радиус оболочки (np.inf - прямая пластина)
plate = 1
plate1 = 0

In [5]:
E = 7e10  # модуль Юнга
ro = 2700  # плотоность алюминия
G = E / (2 * (1 + nu))  # модуль сдвига
L = 1.5  # L1  
W = 1.5   # L2
h = 0.002   # тодщина пластины
D = (E * h ** 3) / (12 * (1 - nu ** 2))  # жесткость из уравнения колебания
N_end = 10  # расчтеные моды
M_end = 10
#te
q1_end = 2   # число шпангаутов
q2_end = 9   # число стрингеров
h_sh = 0.1 # высота шпангоута
h_st = 0.02 # высота стрингера
b_sh_st = 0.002 # толщина сечения стрингера и шпангоута
S = N_end * M_end

In [6]:
if shell == 0 and R < 10000:
    plate1 = 1
    plate = 0
elif shell == 1:
    W = 2 * np.pi * R
    plate = 0
    plate1 = 0

# Считывание нагрузок

In [9]:
siz = 0
siz1 = 0
f0 = np.array([])
p = np.array([])
x0 = np.array([])
Len = np.array([])
L1 = np.array([])
L2 = np.array([])
Uph = np.array([])

# Моменты инерции для шпангоутов

In [10]:
S_sh = h_sh * b_sh_st  # профиль шпангоута
b_pan_sh = L / (q1_end + 1) # пролет шпангаута
I_sh = b_sh_st * (h_sh ** 3) / 12 # момент инерции
Ik_sh = ((b_sh_st ** 3) * h_sh / 3) * (1 - 0.63 * (b_sh_st / h_sh) + 0.052 * (b_sh_st / h_sh) ** 5)
Iro_sh = 0.5 * (b_sh_st * h_sh) * (h_sh ** 2 + b_sh_st ** 2)
S_pan_sh = b_pan_sh * h
V_sh = S_pan_sh * (h / 2 + h_sh / 2) / (S_pan_sh + S_sh)
A_sh = E * (I_sh + S_sh * V_sh ** 2)
ma_sh = ro * S_sh

# Моменты инерции для стрингеров

In [11]:
S_st = h_st * b_sh_st # профиль стрингера
b_pan_st = W / (q2_end + 1) # пролет стрингера
I_st = b_sh_st * (h_st ** 3) / 12
Ik_st = ((b_sh_st ** 3) * h_st / 3) * (1 - 0.63 * (b_sh_st / h_st) + 0.052 * (b_sh_st / h_st) ** 5)
Iro_st = 0.5 * (b_sh_st * h_st) * (h_st ** 2 + b_sh_st ** 2)
S_pan_st = b_pan_st * h
V_st = S_pan_st * (h / 2 + h_st / 2) / (S_pan_st + S_st)
A_st = E * (I_st + S_st * V_st ** 2)
ma_st = ro * S_st

In [12]:
# координаты стрингеров и шпангоутов
x1 = np.arange(1, q1_end + 1) * b_pan_sh
x2 = np.arange(1, q2_end + 1) * b_pan_st

# Задаем пластину

In [13]:
# функции для вычисления ki, kj
def ku1(i):
    return (np.pi * i) / L
def ku2(j):
    return plate * (np.pi * j) / W + plate1 * (np.pi * j) / W + shell * j / R

In [14]:
# задаем матрицу оператора d^4/dx^4
Z0_eig = np.zeros((S, S))
qu0 = np.zeros((S, S))
for i in range(1, M_end + 1):
    for j in range(1 ,N_end + 1):
        k = (i - 1) * N_end + j
        ku = ku1(i) ** 4 + 2 * (ku1(i) * ku2(j)) ** 2 + ku2(j) ** 4
        Z0_eig[k-1][k-1] = D * ku + (plate1 * E * h * ku1(i) ** 4) / (ku * R ** 2) +(shell * E * h * ku1(i) ** 4) / (ku * R ** 2)
        qu0[k-1][k-1] = D * ku + (plate1 * E * h * ku1(i) ** 4) / (ku * R ** 2) + (shell * E * h * ku1(i) ** 4) / (ku * R ** 2)

In [15]:
# задаем массовую матрицу
Z0_eig_m = np.diag([ro * h] * S)

In [16]:
W1 = np.diag([L / 2] * M_end)
W2 = np.diag([W / 2] * N_end)

# Обобщенные силы

In [18]:
x1_end = step
x2_end = step
dx1 = L / step
dx2 = W / step

In [ ]:
U_res = np.zeros((x1_end ** 2 + x2_end, x1_end ** 2 + x2_end))
for i1 in range(1, x1_end + 1):
    for i3 in range(1, x1_end + 1):
        for i2 in range(1, x2_end + 1):
            for i4 in range(1, x2_end + 1):
                ix1 = (i1 - 1) * (x1_end + 1) + i2
                ix2 = (i3 - 1) * (x1_end + 1) + i4
                x1a = (i1 - 1) * dx1
                x2a = (i2 - 1) * dx2
                x3a = (i3 - 1) * dx1
                x4a = (i4 - 1) * dx2
                U_res[ix1][ix2] = exp(-p1)